# JobGPT

Semantic search of LinkedIn job search backed by ChatGPT. For demonstration purpose only.   
**Be aware that web scrapping does directly violate LinkedIn term of service unfortuantely.** 
Also if you do that too fast too often
you run out of OpenAI quota first and either you have to lower your tps or you need to pay huge some for OpenAI apis.

## Log in to LinkedIn
Right now it's done by manual. You log in as how you do it via usual LinkedIn and leave it there.

In [1]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time

#service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome('./chromedriver.exe')
driver.get("https://linkedin.com/uas/login")
time.sleep(5)

C:\Users\Jianghong\AppData\Local\Temp\ipykernel_26280\3546711880.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


## Log in to Google Spreadsheets

This is the sheet you want to use for all future job posting store. You can get the `SPREADSHEET_ID` from your spreadsheet url:`https://docs.google.com/spreadsheets/d/your_spreadsheet_id/edit#gid=961703489`. Copy it from your link and save it to the variable below.

In [2]:
## IMPORTANT: THIS IS YOUR GOOGLE SPREADSHEET ID, which is the sheet

SPREADSHEET_ID = '1_uF0kZgWKjo9JhWoCFSLeRIZmZweCI0OYPDgNGdQEmE'   

In [3]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

#
 
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

token_path = 'token.json'
if os.path.exists(token_path):
    creds = Credentials.from_authorized_user_file(token_path, SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)


In [4]:
def create_tab(service, spreadsheet_id, tab_name):

    # Get the spreadsheet information
    spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()

    # Get the list of sheets in the spreadsheet
    sheets = spreadsheet.get('sheets', [])
    for sheet in sheets:
        if sheet['properties']['title'] == tab_name:
            print('Sheet:' + tab_name + ' exists')
            return
    else:
        print('create new sheet')
    # Set the properties for the new sheet      

    sheet_properties = {
        'title': tab_name,
        'gridProperties': {
            'rowCount': 300,
            'columnCount': 20
        }
    }

    # Create the request to add the new sheet
    request = {
        'addSheet': {
            'properties': sheet_properties
        }
    }

    # Execute the request to create the new sheet
    response = service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={'requests': [request]}).execute()

    # Get the ID of the new sheet
    sheet_id = response['replies'][0]['addSheet']['properties']['sheetId']

    data = [['Date','Link','Company JD','Visa sponsorship','Tech stack','YoE',
    'salary','application status']]
    # write header:
    service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=f'{tab_name}!A1',
        valueInputOption='USER_ENTERED',
        body={'values': data}
    ).execute()


Create new tab for each new semantic search.

In [5]:
from datetime import date
today = date.today().strftime("%Y%m%d")

#Feel free to change tab_name to your liking
tab_name = 'Apple'

create_tab(service=service, spreadsheet_id=SPREADSHEET_ID, tab_name=tab_name)

Sheet:Apple exists


In [6]:
def get_job_description(page_source):
    src = driver.find_elements(By.CLASS_NAME  ,'jobs-box__html-content')[0].get_attribute('innerHTML')
    import html2text
    markdown_text = html2text.html2text(src)
    return markdown_text
    #print(markdown_text)

src

In [8]:
def get_company(page_source):
# print(dom.xpath('/html/body/div[5]/div[3]/div[4]/div/div/main/div/section[2]/div/div[2]/div[1]/div/div[2]/article/div/div[1]'))   
    return driver.find_elements(By.CLASS_NAME,'jobs-unified-top-card__company-name')[0].find_elements(By.TAG_NAME, 'a')[0].get_attribute('href')

## Setup OpenAI for langchain.

In [17]:
from selenium.webdriver.common.by import By

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
import os
import json
api_key = ''
with open('openai.json','r') as secret:
    api_key = json.load(secret)['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = api_key
import string_loader
from bs4 import BeautifulSoup

NameError: name 'importlib' is not defined

#

In [11]:
from typing import List

def multi_query(doc: str, questions: List[str]) -> List[str]:
    loader = string_loader.StringLoader(doc)
    input_documents = loader.load()
    chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
    res = []
    for question in questions:
        res.append(chain.run(input_documents=input_documents, question=question))
    return res

In [12]:
from selenium.common.exceptions import NoSuchElementException

def reach_end_of_search(driver) -> bool:
    try:
        e = driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div[4]/div/div[1]/div/h1')
        return e.text == 'No matching jobs found.'
    except NoSuchElementException as e:
        return False

In [13]:
def update_spreadsheet(service, sheet_name:str, tab_name:str, row: List[str],row_number):
    import gspread
    '''
    Update google spreadsheet sheet_name at tab_name and at row: row_number, starting from column A
    '''
    service.spreadsheets().values().update(
        spreadsheetId=sheet_name,
        # If you want to add more column, make sure you update range accordingly.
        range=f'{tab_name}!A{row_number}:H{row_number}',
        valueInputOption='USER_ENTERED',
        body={'values': [row]}
    ).execute()
    return row_number + 1

In [14]:
def get_job_link(url):
    import urllib.parse
    #TODO: right now the job link doesn't work properly. I believe it's due to lack
    # of other parameters in url so it triggers LinkedIn's anti bot safety measure.
    # You can bypass by copy pasting job id aka the part after /views/ into a working
    # job link, e.g. 
    return 'https://www.linkedin.com/jobs/views/' + urllib.parse.parse_qs(url.split('?')[1])['currentJobId'][0]

In [19]:
import string_loader
# These are the the question you want to get answers from job description
QUERIES = ['Does the job sponsor h1b?', 'What\'re the tech stacks required?', 'How many years of experience is required?',
           'What\'s the maximum salary for the job?']

# For each new search you want to start with 2, since row 1 is header.
# If you want to resume last search, you can change it to the top available row.
spreadsheet_last_row = 2

# This is the upper limit per each sematic search.
UPPER_LIMIT = 5000

# To simplify things, I choose not to use selenium to click on LinkedIn filters.
# Instead just set the search criterions until you're satisfied, and paste it here.
# Make sure you're at page1.
LINKED_IN_JOB_SEARCH_URL = 'https://www.linkedin.com/jobs/search/?currentJobId=3486463613&f_C=162479&f_JT=F&f_TPR=r604800&keywords=software%20engineer&sortBy=R'

# If you want to resume last search, you can change range to (last_index, UPPER_LIMIT, 25)
# 25 is a hard-coded LinkedIn job descriptions per list.
for starting_index in range(0, UPPER_LIMIT, 25):
    driver.get(LINKED_IN_JOB_SEARCH_URL + '&start=' + str(starting_index))
    if reach_end_of_search(driver):
        print('Reaching end of job search at index:' + starting_index)
        break

    for i in range(25):
        try:
            job_listings = driver.find_elements(By.CLASS_NAME, "job-card-container")
            job = job_listings[i]
            job.click()
            jd = get_job_description(driver.page_source)
            company = get_company(driver.page_source)
            answers_list = multi_query(jd, QUERIES)
            spreadsheet_last_row = update_spreadsheet(service, SPREADSHEET_ID, tab_name, [date.today().strftime(
                "%Y%m%d"), get_job_link(driver.current_url), company, answers_list[0], answers_list[1], answers_list[2], answers_list[3]], spreadsheet_last_row)
            
            # What this does is it gradually scroll down the job list, each time put the current job
            # to the top, so browser will keep load next jobs.
            driver.execute_script("arguments[0].scrollIntoView();", job)

            # Halt the scrapping for 10 sec so you don't hit ChatGPT quota and the bot is benign/inefficient enough.
            time.sleep(10)
        except Exception as e:
            print(e)
            print(get_job_link(driver.current_url))

        
    # driver.execute_script('window.scrollTo(0,document.)')
        


Message: stale element reference: element is not attached to the page document
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x002D37D3]
	(No symbol) [0x00268B81]
	(No symbol) [0x0016B36D]
	(No symbol) [0x0016E0FB]
	(No symbol) [0x0016DFD0]
	(No symbol) [0x0016EA35]
	(No symbol) [0x001D209F]
	(No symbol) [0x001BB41C]
	(No symbol) [0x001D1215]
	(No symbol) [0x001BB216]
	(No symbol) [0x00190D97]
	(No symbol) [0x0019253D]
	GetHandleVerifier [0x0054ABF2+2510930]
	GetHandleVerifier [0x00578EC1+2700065]
	GetHandleVerifier [0x0057C86C+2714828]
	GetHandleVerifier [0x00383480+645344]
	(No symbol) [0x00270FD2]
	(No symbol) [0x00276C68]
	(No symbol) [0x00276D4B]
	(No symbol) [0x00280D6B]
	BaseThreadInitThunk [0x766000F9+25]
	RtlGetAppContainerNamedObjectPath [0x76FC7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76FC7B8E+238]
	(No symbol) [0x00000000]

https://www.linkedin.com/jobs/views/3486463613


Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID f78a9567ac66e9056ac35368ef2220d6 in your message.).


<HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1_uF0kZgWKjo9JhWoCFSLeRIZmZweCI0OYPDgNGdQEmE/values/Apple%21A301%3AH301?valueInputOption=USER_ENTERED&alt=json returned "Range (Apple!A301:H301) exceeds grid limits. Max rows: 300, max columns: 20". Details: "Range (Apple!A301:H301) exceeds grid limits. Max rows: 300, max columns: 20">
https://www.linkedin.com/jobs/views/3306871181
<HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1_uF0kZgWKjo9JhWoCFSLeRIZmZweCI0OYPDgNGdQEmE/values/Apple%21A301%3AH301?valueInputOption=USER_ENTERED&alt=json returned "Range (Apple!A301:H301) exceeds grid limits. Max rows: 300, max columns: 20". Details: "Range (Apple!A301:H301) exceeds grid limits. Max rows: 300, max columns: 20">
https://www.linkedin.com/jobs/views/3405051434
<HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1_uF0kZgWKjo9JhWoCFSLeRIZmZweCI0OYPDgNGdQEmE/values/Apple%21A301%3AH301?valueInputOption=USER_ENTERED

Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-zHuVnDGlAhBl2sQZrBiIwZqz on requests per min. Limit: 60.000000 / min. Current: 70.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


<HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1_uF0kZgWKjo9JhWoCFSLeRIZmZweCI0OYPDgNGdQEmE/values/Apple%21A301%3AH301?valueInputOption=USER_ENTERED&alt=json returned "Range (Apple!A301:H301) exceeds grid limits. Max rows: 300, max columns: 20". Details: "Range (Apple!A301:H301) exceeds grid limits. Max rows: 300, max columns: 20">
https://www.linkedin.com/jobs/views/3401287798


Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-zHuVnDGlAhBl2sQZrBiIwZqz on requests per min. Limit: 60.000000 / min. Current: 70.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


KeyboardInterrupt: 